### Importing Required Libraries

In [3]:
import os
import pandas as pd
import getpass # to get the current user
from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationChain
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from pydantic.v1 import BaseModel, Field
from typing import Optional, TypedDict, Annotated
import operator
from semantic_router.utils.function_call import FunctionSchema

### Defining file read tool

In [4]:
def read_python_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Error: File not found."
    
class codepath(BaseModel):
  path: str = Field(description="code path to execute")

@tool(args_schema = codepath)
def execute_query(path: str) -> str:
  """Returns the result of code path execution"""
  return read_python_file(path)

In [5]:
# import requests
# # Function to fetch repository contents from GitHub
# def get_repo_contents(repo_owner, repo_name, access_token):
#     """
#     Fetch the contents of a repository from GitHub.

#     :param repo_owner: Owner of the repository
#     :param repo_name: Name of the repository
#     :param access_token: Personal access token for GitHub
#     :return: Repository contents as a list of files and directories
#     """
#     url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents"

#     headers = {
#         "Authorization": f"token {access_token}",
#         "Accept": "application/vnd.github.v3+json"
#     }
#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         return response.json()
#     else:
#         return f"Error: Unable to fetch repository contents (Status Code: {response.status_code})"

# # Example usage
# repo_owner = "akhilntap"
# repo_name = "ProductionReadinessAgenticAI"
# access_token = os.environ.get("githubpat")
# repo_contents = get_repo_contents(repo_owner, repo_name, access_token)
# print(repo_contents)
# repo_table = pd.DataFrame(repo_contents)
# repo_table

In [6]:
# def fetch_github_file_content(repo_contents, file_name):
#     """
#     Fetch the content of a specific file from the GitHub repository.

#     :param repo_contents: List of repository contents
#     :param file_name: Name of the file to fetch
#     :return: File content as a string or an error message
#     """
#     # Find the file in the repository contents
#     file_info = next((item for item in repo_contents if item['name'] == file_name), None)
    
#     if not file_info:
#         return f"Error: File '{file_name}' not found in the repository."
    
#     if not file_info.get('download_url'):
#         return f"Error: File '{file_name}' does not have a downloadable URL."
    
#     # Fetch the file content
#     response = requests.get(file_info['download_url'])
    
#     if response.status_code == 200:
#         return response.text
#     else:
#         return f"Error: Unable to fetch file content (Status Code: {response.status_code})"

# # Example usage
# file_name = "40poc.ipynb" 
# file_content = fetch_github_file_content(repo_contents, file_name)
# print(file_content)

In [7]:

#when you write code for code diff pull based on pull request, modify this function with good way of representing parameters
def fetch_file_from_github(file_name, repo_owner="akhilntap", repo_name="ProductionReadinessAgenticAI", access_token=os.environ.get("githubpat")):
    """
    Fetch the content of a specific file from a GitHub repository.

    :param repo_owner: Owner of the repository
    :param repo_name: Name of the repository
    :param access_token: Personal access token for GitHub
    :param file_name: Name of the file to fetch
    :return: File content as a string or an error message
    """
    # Fetch repository contents
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents"
    headers = {
        "Authorization": f"token {access_token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return f"Error: Unable to fetch repository contents (Status Code: {response.status_code})"
    
    repo_contents = response.json()

    # Find the file in the repository contents
    file_info = next((item for item in repo_contents if item['name'] == file_name), None)
    
    if not file_info:
        return f"Error: File '{file_name}' not found in the repository."
    
    if not file_info.get('download_url'):
        return f"Error: File '{file_name}' does not have a downloadable URL."
    
    # Fetch the file content
    file_response = requests.get(file_info['download_url'])
    
    if file_response.status_code == 200:
        return file_response.text
    else:
        return f"Error: Unable to fetch file content (Status Code: {file_response.status_code})"
    
class githubfile(BaseModel):
  path: str = Field(description="github file to fetch")

@tool(args_schema = githubfile)
def execute_githubpull(file_name: str) -> str:
  """Pull the code contents from github"""
  return fetch_file_from_github(file_name)

### defining agent state

In [8]:
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], operator.add]

In [9]:
# create the function calling schema for ollama
execute_query_schema = FunctionSchema(read_python_file).to_ollama()
# execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema["function"]["parameters"]["properties"]["description"] = None
execute_query_schema

{'type': 'function',
 'function': {'name': 'read_python_file',
  'description': 'None',
  'parameters': {'type': 'object',
   'properties': {'file_path': {'description': None, 'type': 'object'},
    'description': None},
   'required': []}}}

In [ ]:
class codeAgent:
  # initialising the object
  def __init__(self, model, tools, system_prompt = ""):
    self.system_prompt = system_prompt

    # initialising graph with a state 
    graph = StateGraph(AgentState)

    # adding nodes 
    graph.add_node("llm", self.call_llm)
    graph.add_node("function", self.execute_function)
    graph.add_conditional_edges(
      "llm",
      self.exists_function_calling,
      {True: "function", False: END}
    )
    graph.add_edge("function", "llm")

    # setting starting point
    graph.set_entry_point("llm")

    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def call_llm(self, state: AgentState):
    messages = state['messages']
    # adding system prompt if it's defined
    if self.system_prompt:
        messages = [SystemMessage(content=self.system_prompt)] + messages

    # calling LLM
    message = self.model.invoke(messages)

    return {'messages': [message]}
  
  def execute_function(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls

    results = []
    for tool in tool_calls:
      # checking whether tool name is correct
      if not tool['name'] in self.tools:
        # returning error to the agent 
        result = "Error: There's no such tool, please, try again" 
      else:
        # getting result from the tool
        result = self.tools[tool['name']].invoke(tool['args'])

      results.append(
        ToolMessage(
          tool_call_id=tool['id'], 
          name=tool['name'], 
          content=str(result)
        )
    )
    return {'messages': results}
  
  def exists_function_calling(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0


# from langchain_ollama import ChatOllama
# model = ChatOllama(model="llama3.2:1b")
# # model = ChatOllama(model="codellama:latest")

# system prompt
# prompt = '''You are a senior expert in reviewing python code. 
# So, you can help the team to review the code and provide feedback. 
# You are very accurate and take into account all the nuances in code.
# Your goal is to provide the detailed documentation for any security issues in the code that will help users.'''

# prompt = '''You are a senior expert in reviewing code for observability. The best one that exists.
# Your goal is to analyze the code on the following questions 
# 1. Are there actionable alerts identified for the feature? Are there Runbooks for the actionable alerts? Do we have TSGs attached to the alert?
# 2. Add metrics to monitor dependencies and exception handling on components, infrastructure and features so that SRE can create alerts to reduce TTD?
# 3. Are there CorrelationIDs established in logs to derive error lineage across various components?
# 4. Can the feature/service support changing log levels for troubleshooting purposes?
# 5. Are there critical log lines that we need to get alerted upon?
# Provide response in the format as follows: {question: response}
# '''

prompt = '''You are a senior expert in reviewing code for resiliency. The best one that exists.
Your goal is to analyze the code on the following questions 
1. Can the service/feature sustain a single node/pod failure?
2. Can the feature/service recover gracefully without requiring a restart?
3. Are there conditions that will require a human intervention to start the pods? if so, what are they?
4. What will be the customer impact(if any) due to lack of high availability?
Provide response in the format as follows: {question: response}
'''

import os
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

model = AzureAIChatCompletionsModel(
    endpoint= os.getenv('azureaifoundrymistralendpoint'),
    credential=os.getenv('azureaifoundrymistralkey'),
    model="mistralai-mistral-7b-instruc-11",
)
doc_agent = codeAgent(model, [execute_query], system_prompt=prompt)



import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "input_data": {
    "input_string": [
      {
        "role": "user",
        "content": "I am going to Paris, what should I see?"
      },
      {
        "role": "assistant",
        "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
      },
      {
        "role": "user",
        "content": "What is so great about #1?"
      }
    ],
    "parameters": {
      "temperature": 0.8,
      "top_p": 0.8,
      "max_new_tokens": 4096
    }
  }
}

body = str.encode(json.dumps(data))

url =os.getenv('azureaifoundrymistralendpoint')
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = os.getenv('azureaifoundrymistralkey')
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

# messages = [HumanMessage(content="the code is in the path '/Users/akhilred/Desktop/Billing Pyton Script/Billing_Usage_Extraction.py'. Analyze the code for resiliency requirements mentioned in the prompt")]
messages = [HumanMessage(content="Analyze the code with name 40poc.ipynb from github for resiliency requirements mentioned in the prompt")]
result = doc_agent.graph.invoke({"messages": messages})

2025-04-24 23:40:12 - azure.core.pipeline.policies.http_logging_policy - INFO - _universal.py:509 - on_request() - Request URL: 'https://prragenticworkflow-hlqqu.westus.inference.ml.azure.com/score/info?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'c1ce03b2-2197-11f0-bf34-52304e248078'
    'api-key': 'REDACTED'
    'User-Agent': 'langchain-azure-ai azsdk-python-ai-inference/1.0.0b9 Python/3.13.0 (macOS-15.4.1-arm64-arm-64bit-Mach-O)'
    'Authorization': 'REDACTED'
No body was attached to the request
2025-04-24 23:40:12 - azure.core.pipeline.policies.http_logging_policy - INFO - _universal.py:545 - on_response() - Response status: 424
Response headers:
    'date': 'Fri, 25 Apr 2025 05:40:12 GMT'
    'server': 'azureml-frontdoor'
    'content-length': '22'
    'content-type': 'application/json'
    'x-request-id': 'REDACTED'
    'ms-azureml-model-error-reason': 'REDACTED'
    'ms-azureml-model-error-statuscod

HttpResponseError: Operation returned an invalid status 'Failed Dependency'
Content: {"detail":"Not Found"}

In [ ]:
import os
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

model = AzureAIChatCompletionsModel(
    endpoint= os.getenv('azureaifoundrymistralendpoint'),
    credential=os.getenv('azureaifoundrymistralkey'),
    model="mistralai-mistral-7b-instruc-11",
)
doc_agent = codeAgent(model, [execute_query], system_prompt=prompt)



import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "input_data": {
    "input_string": [
      {
        "role": "user",
        "content": "I am going to Paris, what should I see?"
      },
      {
        "role": "assistant",
        "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
      },
      {
        "role": "user",
        "content": "What is so great about #1?"
      }
    ],
    "parameters": {
      "temperature": 0.8,
      "top_p": 0.8,
      "max_new_tokens": 4096
    }
  }
}

body = str.encode(json.dumps(data))

url =os.getenv('azureaifoundrymistralendpoint')
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = os.getenv('azureaifoundrymistralkey')
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

In [11]:
print(result['messages'][-1].content)

NameError: name 'result' is not defined

### Security Vulnerabilities using bandit

In [12]:
from bandit.core import manager, config

def check_code_security(file_path):
    """
    Check a Python file for security vulnerabilities using Bandit.

    :param file_path: Path to the Python file to analyze
    :return: Bandit report as a string
    """
    # Load Bandit configuration
    bandit_config = config.BanditConfig()

    # Initialize Bandit manager
    bandit_manager = manager.BanditManager(bandit_config, "file", False)

    # Run Bandit on the specified file
    bandit_manager.discover_files([file_path])
    bandit_manager.run_tests()

    # Generate and return the report
    issues = bandit_manager.get_issue_list()
    if not issues:
        return "No high-level vulnerabilities found. Checking for lower-level issues..."

    # If no high-level issues, check for lower-level issues
    lower_level_issues = []
    for issue in issues:
        if issue.severity.lower() in ['low', 'medium']:
            lower_level_issues.append(str(issue))

    if lower_level_issues:
        return "\n".join(lower_level_issues)
    else:
        return "No vulnerabilities found, including lower-level issues."

# test
file_path = "/Users/akhilred/Desktop/ProductionReadinessAgenticAI/vulntest.py"
print(check_code_security(file_path))

2025-04-24 23:40:22 - stevedore.extension - ERROR - extension.py:227 - _load_plugins() - Could not load 'sarif': No module named 'sarif_om'


No high-level vulnerabilities found. Checking for lower-level issues...


In [5]:
# import os
# from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint
# from langchain_community.chat_models.azureml_endpoint import (
#     AzureMLEndpointApiType,
#     CustomOpenAIChatContentFormatter,
# )
# from langchain_core.messages import HumanMessage

# chat = AzureMLChatOnlineEndpoint(
#     # endpoint_url=os.getenv('azureaifoundrymistralendpoint'),
#     endpoint_url= "https://prragenticworkflow-hlqqu.westus.inference.ml.azure.com/score",
#     endpoint_api_type=AzureMLEndpointApiType.dedicated,
#     endpoint_api_key=os.getenv('azureaifoundrymistralkey'),
#     content_formatter=CustomOpenAIChatContentFormatter(),
# )
# response = chat.invoke(
#     [HumanMessage(content="Will the Collatz conjecture ever be solved?")]
# )
# response

In [4]:
import urllib.request
import json

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "input_data": {
    "input_string": [
      {
        "role": "user",
        "content": "I am going to Paris, what should I see?"
      },
      {
        "role": "assistant",
        "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
      },
      {
        "role": "user",
        "content": "What is so great about #1?"
      }
    ],
    "parameters": {
      "temperature": 0.8,
      "top_p": 0.8,
      "max_new_tokens": 4096
    }
  }
}

body = str.encode(json.dumps(data))

url =os.getenv('azureaifoundrymistralendpoint')
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = os.getenv('azureaifoundrymistralkey')
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))
 

b'{"output":"The Eiffel Tower is an iconic symbol of Paris and one of the most recognizable landmarks in the world. Here are some reasons why it\'s so great:\\n\\n1. **Engineering marvel**: When it was built for the 1889 World\'s Fair, the Eiffel Tower was a technological marvel. It was the tallest man-made structure in the world at the time, standing at 324 meters (1,063 feet) high. Its innovative design and engineering made it possible to build a structure that was both strong and elegant.\\n2. **Breathtaking views**: The Eiffel Tower offers stunning views of the city from its observation decks. On a clear day, you can see for miles in every direction, taking in the beautiful architecture, gardens, and streets of Paris.\\n3. **Romantic ambiance**: The Eiffel Tower is a popular spot for couples and proposals. The tower\'s iron latticework and beautiful gardens create a romantic atmosphere, making it a perfect spot for a date or a special moment.\\n4. **History and significance**: The 